<a href="https://colab.research.google.com/github/rtajeong/M1_2025/blob/main/Ch6_Scraping_rev1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 크롤링 예제

In [ ]:
import requests
from bs4 import BeautifulSoup

def simple_crawler(url):
    response = requests.get(url)
    response.raise_for_status()  # http error 발생시 예외 발생

    soup = BeautifulSoup(response.text, 'html.parser')

    links = soup.find_all('a')   # 모든 <a> 태그(링크) 찾기

    if links:
        for link in links:
            href = link.get('href')  # 링크의 'href' 속성 값 가져오기
            if href:
                print(href)

start_url = "https://www.google.com" # 크롤링 시작 웹페이지 URL(예: 구글 홈, 네이버 홈)
simple_crawler(start_url)

https://www.google.com/imghp?hl=en&tab=wi
https://maps.google.com/maps?hl=en&tab=wl
https://play.google.com/?hl=en&tab=w8
https://www.youtube.com/?tab=w1
https://news.google.com/?tab=wn
https://mail.google.com/mail/?tab=wm
https://drive.google.com/?tab=wo
https://www.google.com/intl/en/about/products?tab=wh
http://www.google.com/history/optout?hl=en
/preferences?hl=en
https://accounts.google.com/ServiceLogin?hl=en&passive=true&continue=https://www.google.com/&ec=GAZAAQ
/advanced_search?hl=en&authuser=0
/intl/en/ads/
/services/
/intl/en/about.html
/intl/en/policies/privacy/
/intl/en/policies/terms/


#  Scraping: 정적 페이지에서 명언 정보 추출

In [4]:
import requests
from bs4 import BeautifulSoup

url = "http://quotes.toscrape.com/"
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

quotes = soup.select("div .quote")  # class="quote"
print (f"There are {len(quotes)} quotes on the page.")

for num, quote in enumerate(quotes, 1):
    text = quote.select_one(".text").text
    author = quote.select_one(".author").text
    print(f"{num}. {text} - {author}")


There are 10 quotes on the page.
1. “The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.” - Albert Einstein
2. “It is our choices, Harry, that show what we truly are, far more than our abilities.” - J.K. Rowling
3. “There are only two ways to live your life. One is as though nothing is a miracle. The other is as though everything is a miracle.” - Albert Einstein
4. “The person, be it gentleman or lady, who has not pleasure in a good novel, must be intolerably stupid.” - Jane Austen
5. “Imperfection is beauty, madness is genius and it's better to be absolutely ridiculous than absolutely boring.” - Marilyn Monroe
6. “Try not to become a man of success. Rather become a man of value.” - Albert Einstein
7. “It is better to be hated for what you are than to be loved for what you are not.” - André Gide
8. “I have not failed. I've just found 10,000 ways that won't work.” - Thomas A. Edison
9. “A woman is like a tea bag; you never kn

# 오픈 API (기상청 단기 예보)
- 많은 공공기관 또는 민간 서비스는 오픈 API를 통해 구조화된 데이터를 제공한다. HTML을 분석하지 않아도 되므로 안정적이고 정확하게 원하는 정보를 얻을 수 있다는 장점이 있다. 특히 공공데이터 포털, 기상청, 뉴스, 금융 등의 분야에서 매우 유용하게 활용되고 있다.
- 오픈 API는 일반 사용자에게 공개된 API로, 보통 인증키 발급을 통해 사용할 수 있다.
- 대부분의 오픈 API는 JSON 또는 XML 형식으로 데이터를 제공한다.

## 공공데이터 포털(https://www.data.go.kr) 에서 인증키 발급받기
1. 공공데이터 포털 회원가입 및 로그인 후 인증키를 발급받는다.
2. 원하는 공공데이터 검색 및 활용 신청
   - 데이터 검색: 포털의 검색창을 통해 원하는 공공데이터를 검색한다. 예를 들어, "날씨", "미세먼지", "버스", "지하철" 등으로 검색
   - 검색 결과 중 Open API 탭에서 원하는 데이터셋을 찾아 클릭.
   - 선택한 API 상세 페이지에서 "활용 신청" 버튼을 클릭
3. 인증키(Service Key) 발급 및 확인
   - 활용 신청이 완료되면 마이페이지에서 "API 키 발급/관리" 또는 "인증키 발급 현황" 메뉴에서 발급된 인증키를 확인 가능
   - 인증키는 일반적으로 인코딩된(Encoded) 버전과 디코딩된(Decoded) 버전이 제공되는데, API 호출 시 (특별히 지정되어 있지 않으면) 인코딩된 키를 주로 사용한다.
4. API 명세 확인 및 테스트
   - 각 Open API 상세 페이지에는 해당 API를 호출하는 데 필요한 요청 변수(Request Parameters), 응답 형식(Response Format), 샘플 URL 등이 됨.
   - 샘플 URL 제공됨.

## params 딕셔너리를 이용한 API 호출
- 이 방식은 API 요청의 파라미터들을 딕셔너리 형태로 관리하며 requests 라이브러리가 URL을 구성하게 하는 일반적인 방법 (키의 숨겨진 공백 문제를 해결하기 위해 my_key.strip()을 적용)
- 특정 환경이나, 서비스 키 문자열에 눈에 보이지 않는 미세한 공백 또는 특수 문자가 포함되어 있을 경우, 파이썬 코드에서는 키가 유효하지 않다고 인식되어 SERVICE_KEY_IS_NOT_REGISTERED_ERROR 오류 메시지를 받을 수 있다.
- (주의) 이 사이트는 하루 정보만을 제공하기 때문에 실행 전 날짜 정보 (base_date)를 고쳐 주어야 한다.

In [ ]:
import requests
import pandas as pd

# 1. API 호출 정보 설정
# 주의: 복사-붙여넣기 시 앞뒤 공백이 포함되지 않도록 .strip()을 사용한다.
my_key="UrcOXfOckvYo8hX%2F8AmlgQ8nIUSgSPH1u%2BuX7Z%2BkObecxxvDE8dHnkY3LQH2%2FPfOy8%2FM4LBA7dwd4eVpRTyQ4g%3D%3D".strip()
# my_key = "UrcOXfOckvYo8hX%2F8AmlgQ8nIUSgSPH1u%2BuX7Z%2BkObecxxvDE8dHnkY3LQH2%2FPfOy8%2FM4LBA7dwd4eVpRTyQ4g%3D%3D".strip()
base_url = "http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getUltraSrtNcst"

# API 요청에 필요한 파라미터들을 딕셔너리 형태로 정의한다 (해당 사이트의 샘플 코드 참고)
params = {
    'serviceKey': my_key,
    'pageNo': '1',
    'numOfRows': '10',
    'dataType': 'JSON',
    'base_date': '20250710', # 예시 날짜 (API 명세에 맞는 형식)
    'base_time': '0600',    # 예시 시간 (API 명세에 맞는 형식)
    'nx': '55', # 예시 지점의 X좌표 (기상청 격자 좌표)
    'ny': '127' # 예시 지점의 Y좌표 (기상청 격자 좌표)
}

# 2. API 요청 및 응답 처리
# requests 라이브러리가 base_url과 params 딕셔너리를 조합하여 최종 URL을 구성하고 요청한다.
response = requests.get(base_url, params=params)

# 3. HTTP 상태 코드 확인 및 데이터 파싱
if response.status_code == 200:
    # 응답이 성공이면 파싱
    json_data = response.json()
    # 필요한 데이터는 'response' > 'body' > 'items' > 'item' 경로에 있다.
    items = json_data['response']['body']['items']['item']
    # Pandas DataFrame으로 변환하여 데이터를 표 형태로 정리한다.
    df = pd.DataFrame(items)
    print("API 호출 성공 및 DataFrame 생성 완료")
    print(df)
else:
    # 응답이 실패(200이 아님)한 경우 상태 코드를 출력한다.
    print(f"API 요청 실패: HTTP 상태 코드 {response.status_code}")
    print(f"서버 응답 내용: \n{response.text}") # 실패 시 서버가 보낸 메시지 확인

API 요청 실패: HTTP 상태 코드 404
서버 응답 내용: 
<!DOCTYPE html>
<html><head><meta http-equiv="Content-Type" content="text/html; charset=UTF-8">
<title>ìì¤í ì ê²</title>
<meta http-equiv="X-UA-Compatible" content="IE=edge">
<meta name="viewport" content="width=device-width, initial-scale=1, maximum-scale=1, user-scalable=no">
<meta name="author" content="êµ­ê°ì ë³´ììê´ë¦¬ì">
<!-- <link rel="shortcut icon" href="/favicon.ico"> -->
<style>
html,body { width:100%; height:100%; margin:0; padding:0; font-size:1em; line-height:1.5; color:#666; font-family: "Noto Sans", "Malgun Gothic", "ë§ìê³ ë", "ëì" }
.wrapper { width:100%; height:100%; position:relative; }


.PZwindow {
	border:1px solid #ddd;
	background-color:#fff;
    position:absolute;
	/*position:fixed !important;*/
    z-index:99999;
}
.PZwindow * { 
	font-family: "NotoKR", "Noto Sans", "Malgun Gothic", "ë§ìê³ ë", "ëì" !important;
	font-size:1em; 
	line-height:1.5; 
	color:#2e2e38;
	letter-spacing:-0.05em; 
}
.

## full_url을 이용한 API 호출
- 브라우저에서 직접 테스트하여 완벽하게 작동하는 전체 URL 문자열을 직접 사용한다.

In [ ]:
import requests
import pandas as pd

# 1. API 호출 정보 설정 (예시 날짜/시간 고정 :2025년7월10일 06:00 시)
my_key = "UrcOXfOckvYo8hX%2F8AmlgQ8nIUSgSPH1u%2BuX7Z%2BkObecxxvDE8dHnkY3LQH2%2FPfOy8%2FM4LBA7dwd4eVpRTyQ4g%3D%3D".strip()
base_url = "http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getUltraSrtNcst"

# 모든 파라미터가 포함된 완전한 URL
full_api_url = base_url + \
    "?serviceKey=" + my_key + \
    "&pageNo=1&numOfRows=10&dataType=JSON" + \
    "&base_date=20250711&base_time=0600" + \
    "&nx=55&ny=127"

# 2. API 요청 및 응답 처리
response = requests.get(full_api_url)

# 3. 상태 코드 확인 및 DataFrame으로 변환
if response.status_code == 200:
    json_data = response.json()
    # 'response' -> 'body' -> 'items' -> 'item' 경로로 실제 데이터 접근
    items = json_data['response']['body']['items']['item']
    df = pd.DataFrame(items)
    print("API 호출 성공 및 DataFrame 생성 완료 (full_url 방식):")
    print(df)
else:
    print(f"API 요청 실패: HTTP 상태 코드 {response.status_code}")

API 요청 실패: HTTP 상태 코드 404


In [ ]:
json_data

# 동적 스크래핑

동적인 웹페이지는 크게 두 가지 경로로 "새로운 데이터나 화면 변화"가 발생할 수 있다. 즉, DOM 이 바뀌게 된다.
- DOM (Document Object Model) 은 HTML이나 XML 문서를 브라우저가 객체(트리 구조)로 표현한 것. (즉, F12 로 보이는 문서 모델)

(1) AJAX 요청을 통한 서버 통신:
  - 브라우저에서 JavaScript가 비동기 요청(AJAX, fetch, axios 등) 을 서버로 보냄
  - 서버가 JSON, XML, HTML 조각 등을 응답 → 클라이언트가 DOM 일부를 갱신
  - 예: 무한 스크롤 뉴스 피드, 댓글 새로고침 없이 추가
  - scraping: 원본 소스 API 를 찾아 직접 가져오는 방법 이용.

(2) 클라이언트 측 스크립트 실행만으로 변화
  - 서버와 통신하지 않고, 이미 로드된 데이터나 DOM을 조작해서 화면이 바뀌는 경우
  - 브라우저 안에서 JavaScript가 직접 DOM을 수정
  - scraping: 실행 후 DOM 이 변한 후 scraping. (DOM 에 직접 반영이 안되는 경우는 다른 방법 필요)

## Requests 라이브러리로 시도하기: 한계점

In [5]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

url = "http://quotes.toscrape.com/scroll"

response = requests.get(url)
if response.status_code != 200:
    print(f"Error: Failed to fetch the page. Status code: {response.status_code}")
    exit()

soup = BeautifulSoup(response.text, 'html.parser')

# Find all 'div' tags with the class 'quote'.
# In this specific dynamic page's initial HTML, these might not be present.
extracted_data = []
quotes = soup.find_all('div', class_='quote')

if quotes:
    print(f"Found {len(quotes)} quotes (requests - initial load):")
    for quote_div in quotes:
        # Find the quote text (span tag with class='text') within each quote div.
        text_element = quote_div.find('span', class_='text')
        text = text_element.get_text(strip=True) if text_element else 'N/A'

        # Find the author's name (small tag with class='author') within each quote div.
        author_element = quote_div.find('small', class_='author')
        author = author_element.get_text(strip=True) if author_element else 'N/A'

        extracted_data.append({'Quote': text, 'Author': author})
        print(f"- {text[:50]}... (by {author})") # Print example output
else:
    print("Error: Could not find quote elements (div class='quote') in the HTML.")
    print("This confirms the content is dynamically loaded by JavaScript.")

if extracted_data:
    df_requests = pd.DataFrame(extracted_data)
    print("\nQuotes data fetched by requests (DataFrame, first 5 rows):")
    print(df_requests.head())
    print(f"\nTotal quotes fetched by requests: {len(df_requests)}.")
else:
    print("No data extracted by requests.")

Error: Could not find quote elements (div class='quote') in the HTML.
This confirms the content is dynamically loaded by JavaScript.
No data extracted by requests.


- request 가 실패하는 이유:
  - 이 스크립트가 로드되는 최초 HTML(requests가 받아오는 내용)에는 <div class="quote">와 같은 명언 태그가 거의 없다. (또는 아주 초기 몇 개만 있을 수 있다. quotes.toscrape.com/scroll의 경우, 이 스크립트가 첫 10개의 명언도 API로 가져와 append 한다. 즉, 'requests'가 받은 HTML에는 명언 태그가 하나도 없다.)
  - requests는 단순히 response.text에 담긴 HTML 문자열과 JavaScript 코드 자체만 가져올 뿐이고, 브라우저처럼 이 JavaScript 코드를 실행시키지 않는다. 따라서 JavaScript가 API를 호출하고 HTML을 동적으로 생성하여 append하는 과정이 requests에게는 보이지 않는다.
  - response.text에 script 코드가 보이고 명언 태그가 보이지 않는다는 것이 정확히 이 동적 페이지의 특징이며, requests의 한계를 명확하게 보여주는 증거이다.

## 네트워크 트래픽 분석을 통한 원본 데이터(API) 스크래핑

- Network 탭에서 Fetch/XHR 필터(또는 XHR 필터)를 적용한다. 이 필터는 JavaScript가 비동기적으로 데이터를 가져오는 요청들(API 호출)만 걸러준다.
- 나타난 요청들을 클릭해 보면서 Headers 탭에서 요청 URL을 확인하고, Preview 탭이나 Response 탭에서 어떤 데이터가 오는지 확인한다. JSON 또는 XML 형태로 깔끔하게 정돈된 데이터가 보인다면 그 요청이 바로 원본 데이터 소스일 가능성이 높다.
- 특히 URL에 api, data, list, search 등의 키워드가 포함되어 있거나, .json, .xml 확장자를 가진 요청을 찾아본다.

```
Network tab analysis 를 통해 찾은 url: http://quotes.toscrape.com/api/quotes?page=1
```

In [6]:
import requests
import pandas as pd

# The API URL discovered through network tab analysis
# base_api_url = "http://quotes.toscrape.com/api/quotes?page=1"
base_api_url = "http://quotes.toscrape.com/api/quotes"

all_quotes_data_api = [] # List to store all fetched quote data
page_num = 1             # Starting page number for the API

while True:
    print(f"Fetching API page {page_num} data...")
    params = {
        'page': page_num
    }

    response = requests.get(base_api_url, params=params)

    # Check if the HTTP status code is not 200 (OK).
    if response.status_code != 200:
        print(f"Error during API request: Status code {response.status_code}")
        break # Exit loop if API request fails

    json_data = response.json()

    # Check the API response structure: quotes list is under the 'quotes' key
    quotes_on_page = json_data.get('quotes', [])

    if not quotes_on_page: # If no more quotes, break the loop
        print(f"No more quotes found on API page {page_num}. Ending crawl.")
        break

    for quote_info in quotes_on_page:
        quote_text = quote_info.get('text')
        author_name = quote_info.get('author', {}).get('name')
        all_quotes_data_api.append({'Quote': quote_text, 'Author': author_name})

    page_num += 1 # Move to the next page

print(f"Total quotes fetched from API: {len(all_quotes_data_api)}")

if all_quotes_data_api:
    df_api = pd.DataFrame(all_quotes_data_api)
    print("\nQuotes data fetched by direct API call (DataFrame, first 20 rows):")
    print(df_api.head(20))
    print(f"\nTotal quotes fetched: {len(df_api)}.")
else:
    print("No quotes data fetched from API.")

Fetching API page 1 data...
Fetching API page 2 data...
Fetching API page 3 data...
Fetching API page 4 data...
Fetching API page 5 data...
Fetching API page 6 data...
Fetching API page 7 data...
Fetching API page 8 data...
Fetching API page 9 data...
Fetching API page 10 data...
Fetching API page 11 data...
No more quotes found on API page 11. Ending crawl.
Total quotes fetched from API: 100

Quotes data fetched by direct API call (DataFrame, first 20 rows):
                                                Quote               Author
0   “The world as we have created it is a process ...      Albert Einstein
1   “It is our choices, Harry, that show what we t...         J.K. Rowling
2   “There are only two ways to live your life. On...      Albert Einstein
3   “The person, be it gentleman or lady, who has ...          Jane Austen
4   “Imperfection is beauty, madness is genius and...       Marilyn Monroe
5   “Try not to become a man of success. Rather be...      Albert Einstein
6   “It is 

- to see how it worked. (let's get information of the pge 10)

In [7]:
params = "page: 10"
response = requests.get(base_api_url, params=params)

In [12]:
len(response.json())

5

In [9]:
response.json().keys()

dict_keys(['has_next', 'page', 'quotes', 'tag', 'top_ten_tags'])

In [13]:
response.json()['quotes']  #.get('quotes') 와 동일

[{'author': {'goodreads_link': '/author/show/9810.Albert_Einstein',
   'name': 'Albert Einstein',
   'slug': 'Albert-Einstein'},
  'tags': ['change', 'deep-thoughts', 'thinking', 'world'],
  'text': '“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”'},
 {'author': {'goodreads_link': '/author/show/1077326.J_K_Rowling',
   'name': 'J.K. Rowling',
   'slug': 'J-K-Rowling'},
  'tags': ['abilities', 'choices'],
  'text': '“It is our choices, Harry, that show what we truly are, far more than our abilities.”'},
 {'author': {'goodreads_link': '/author/show/9810.Albert_Einstein',
   'name': 'Albert Einstein',
   'slug': 'Albert-Einstein'},
  'tags': ['inspirational', 'life', 'live', 'miracle', 'miracles'],
  'text': '“There are only two ways to live your life. One is as though nothing is a miracle. The other is as though everything is a miracle.”'},
 {'author': {'goodreads_link': '/author/show/1265.Jane_Austen',
   'name': 'Jane 

In [14]:
response.json()['quotes'][0]

{'author': {'goodreads_link': '/author/show/9810.Albert_Einstein',
  'name': 'Albert Einstein',
  'slug': 'Albert-Einstein'},
 'tags': ['change', 'deep-thoughts', 'thinking', 'world'],
 'text': '“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”'}

In [15]:
response.json()['quotes'][0].get('author', [])  # []: 찾고자 하는 키가 없을 때 대신 반환하는 기본값

{'goodreads_link': '/author/show/9810.Albert_Einstein',
 'name': 'Albert Einstein',
 'slug': 'Albert-Einstein'}

In [16]:
response.json()['quotes'][0].get('author', []).get('name')

'Albert Einstein'

In [17]:
response.json()['quotes'][0].get('text')

'“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”'